# Application de la découverte de motifs pour l’analyse de parties de Tennis de Table

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, './emm')
import EMM as emm

In [2]:
# import data
data = pd.read_csv("TT_game.csv", sep = ";",
                   dtype={
                          "Nom": str,
                          "Position": str,
                          "Durée": str,
                          "Genre": str,
                          "Latéralité": str,
                          "Set": str,
                          "Système": str,
                          "Coup": str,
                          "Nbre de coups par point": np.float64,
                          "Durée du point": str,
                          "Score": str,
                          "Points F": np.float64,
                          "Points B": np.float64,
                          "Type de service": str,
                          "Type Coup TK": str,
                          "Zone de jeu": str
})

In [3]:
# convert to deltatime
data['Position'] = pd.to_timedelta(data['Position'])
data['Durée'] = pd.to_timedelta(data['Durée'])

In [4]:
# convert deltatime format to float
data["Position"] = (data["Position"] / np.timedelta64(10**9,'ns')).astype(float)
data["Durée"] = (data["Durée"] / np.timedelta64(10**9,'ns')).astype(float)

In [5]:
# clean "Set" column
for index, row in data.iterrows():
    val = row["Nom"]
    if val.startswith("SET"):
        set_nb = val.split("(")[1][:1]
    data.at[index, "Set"] = set_nb

In [6]:
# data.head(20)

In [7]:
columns = ["Duree", "coup_droit_player_1", "coup_droit_player_2", "revers_player_1", "revers_player_2", "Nb_coups", "Set", \
           "Gagnant_ou_pas", "Service", "controle_player_1", "controle_player_2", "attaque_player_1", "attaque_player_2", \
           "poussette_player_1", "poussette_player_2", "M1_envoie", "M1_reception", "M2_envoie", "M2_reception", \
           "M3_envoie", "M3_reception","D1_envoie", "D1_reception", "D2_envoie", "D2_reception", "D3_envoie",\
           "D3_reception", "G1_envoie", "G1_reception", "G2_envoie", "G2_reception", "G3_envoie", \
           "G3_reception", "Gagnant"]

In [8]:
# # total number of pts
# nb_pts = data["Points F"].dropna()
# nb_pts = len(nb_pts)

# # indices of pts row in the original df
# pts_indices = list(data[data["Points F"].notna()].index)
# pts_indices = [x + 1 for x in pts_indices] # to match df index
# pts_indices.insert(0, 0)

In [9]:
# compute the number of hits for each point
c = 0
for index, row in data.iterrows():
    if (row["Coup"] != "F.T.") and (row["Coup"] != "B.T."):
        c = 0
        data.at[index, "Nbre de coups par point"] = c
    else:
        c += 1
        data.at[index, "Nbre de coups par point"] = c
data["Nbre de coups par point"] = pd.to_numeric(data["Nbre de coups par point"]).astype(int)  

In [10]:
# data.head(10)

In [11]:
def pts_time(point):
    """
    Return the elapsed time for the point.
    """
    time = 0.000
    for index, row in point.iterrows():
        if index != 0:
            time += row["Durée"]
    return time

def class_time(time, quantiles):
    """
    Define the time of the point as class.
    """
    if time < quantiles[0]:
        return "short"
    elif time >= quantiles[0] and time <= quantiles[2]:
        return "middle"
    elif time > quantiles[2]:
        return "long"
    
def pts_score(point, player):
    """
    Return a string that define the player that is winning the set.
    '<': player F.T. is winning
    '>': player B.T. is winning
    '=': draw
    """
    if player == "F":
        score_player1 = int(point.iloc[-1]["Points F"])
        score_player2 = int(point.iloc[-1]["Points B"])
    else:
        score_player1 = int(point.iloc[-1]["Points B"])
        score_player2 = int(point.iloc[-1]["Points F"])
        
    if score_player1 < score_player2:
        return "<"
    elif score_player1 > score_player2:
        return ">"
    else:
        return "="
    
def pts_service(point, player):
    """
    Return a string that decode who did the service and the side
    """
    service = point[point["Type de service"].notna()]["Type de service"].tolist()
    try:
        # if a net is done, it keeps only the last service
        if service[-1] == "Serveur Point - 8-4+I216:I223":
            service = service[-2]
        else:
            service = service[-1]
    except:
        service = np.nan # if double fault
    player_c =  point[point["Coup"].notna()]["Coup"].tolist()[0]
    if player == player_c[0]:
        player = "player1"
    else:
        player = "player2"
    service = str(player) + " - " + str(service)
    return service

def point_winner(point, player): 
    """
    Define if the player has won the point or not. Return a boolean.
    """
    win = point.iloc[-1]["Coup"]
    service = pts_service(point, player)
    has_service = service.split(" ")[0] # get the player that have done the service

    if (has_service == "player1" and win.split(" ")[-1] == "-") or (has_service == "player2" and win.split(" ")[-1] == "+"):
        return "No"
    elif (has_service == "player1" and win.split(" ")[-1] == "+") or (has_service == "player2" and win.split(" ")[-1] == "-"):
        return "Yes"

In [12]:
def df_player(player, data):
    """
    player: 'F' or 'B'
    data: dataframe
    """
    player_data = pd.DataFrame(columns=columns) # df initialisation
    
    if player == "F":
        player1 = "F.T."
        player2 = "B.T."
    elif player == "B":
        player1 = "B.T."
        player2 = "F.T." 
    
    # total number of pts
    nb_pts = data["Points F"].dropna()
    nb_pts = len(nb_pts)

    # indices of pts row in the original df
    pts_indices = list(data[data["Points F"].notna()].index)
    pts_indices = [x + 1 for x in pts_indices] # to match df index
    pts_indices.insert(0, 0)

    # vector with time for each point
    times = []                
    for i in range(0, len(pts_indices)):
        try:
            point = data[pts_indices[i]:pts_indices[i+1]] # extract info for one point
        except:
            break # last point analysed
        
        times.append(pts_time(point))
    quantiles = np.quantile(times, [0.25, 0.5, 0.75])
    
    for i in range(0, len(pts_indices)):
        try:
            point = data[pts_indices[i]:pts_indices[i+1]] # extract info for one point
        except:
            break # last point analysed
            
        coup_droit_player1 = len(point[(point["Latéralité"] == "Coup droit") & (point["Nom"].str.startswith(player1))])
        coup_droit_player2 = len(point[(point["Latéralité"] == "Coup droit") & (point["Nom"].str.startswith(player2))])
        revers_player1 = len(point[(point["Latéralité"] == "Revers") & (point["Nom"].str.startswith(player1))])
        revers_player2 = len(point[(point["Latéralité"] == "Revers") & (point["Nom"].str.startswith(player2))])
        time = class_time(times[i-1], quantiles)
        nb_hit = len(point)
        Set = point["Set"].unique()[0]
        score = pts_score(point, player) # if he is winning the set or not
        service = pts_service(point, player)
        controle_player1 = len(point[(point["Type Coup TK"] == "Controle / Résistance") & (point["Nom"].str.startswith(player1))])
        controle_player2 = len(point[(point["Type Coup TK"] == "Controle / Résistance") & (point["Nom"].str.startswith(player2))])
        attaque_player1 = len(point[(point["Type Coup TK"] == "Attaque / Offensif") & (point["Nom"].str.startswith(player1))])
        attaque_player2 = len(point[(point["Type Coup TK"] == "Attaque / Offensif") & (point["Nom"].str.startswith(player2))])
        poussette_player1 = len(point[(point["Type Coup TK"] == "Poussette") & (point["Nom"].str.startswith(player1))])
        poussette_player2 = len(point[(point["Type Coup TK"] == "Poussette") & (point["Nom"].str.startswith(player2))])
        M1_send = len(point[(point["Zone de jeu"] == "M1") & (point["Nom"].str.startswith(player2))])
        M1_reception = len(point[(point["Zone de jeu"] == "M1") & (point["Nom"].str.startswith(player1))])
        M2_send = len(point[(point["Zone de jeu"] == "M2") & (point["Nom"].str.startswith(player2))])
        M2_reception = len(point[(point["Zone de jeu"] == "M2") & (point["Nom"].str.startswith(player1))])
        M3_send = len(point[(point["Zone de jeu"] == "M3") & (point["Nom"].str.startswith(player2))])
        M3_reception = len(point[(point["Zone de jeu"] == "M3") & (point["Nom"].str.startswith(player1))])
        D1_send = len(point[(point["Zone de jeu"] == "D1") & (point["Nom"].str.startswith(player2))])
        D1_reception = len(point[(point["Zone de jeu"] == "D1") & (point["Nom"].str.startswith(player1))])
        D2_send = len(point[(point["Zone de jeu"] == "D2") & (point["Nom"].str.startswith(player2))])
        D2_reception = len(point[(point["Zone de jeu"] == "D2") & (point["Nom"].str.startswith(player1))])
        D3_send = len(point[(point["Zone de jeu"] == "D3") & (point["Nom"].str.startswith(player2))])
        D3_reception = len(point[(point["Zone de jeu"] == "D3") & (point["Nom"].str.startswith(player1))])    
        G1_send = len(point[(point["Zone de jeu"] == "G1") & (point["Nom"].str.startswith(player2))])
        G1_reception = len(point[(point["Zone de jeu"] == "G1") & (point["Nom"].str.startswith(player1))])
        G2_send = len(point[(point["Zone de jeu"] == "G2") & (point["Nom"].str.startswith(player2))])
        G2_reception = len(point[(point["Zone de jeu"] == "G2") & (point["Nom"].str.startswith(player1))])
        G3_send = len(point[(point["Zone de jeu"] == "G3") & (point["Nom"].str.startswith(player2))])
        G3_reception = len(point[(point["Zone de jeu"] == "G3") & (point["Nom"].str.startswith(player1))])
        win = point_winner(point, player)

        player_data = player_data.append({"Duree": time, "coup_droit_player_1": coup_droit_player1, "coup_droit_player_2": coup_droit_player2, \
            "revers_player_1": revers_player1, "revers_player_2": revers_player2, "Nb_coups": nb_hit, "Set": Set, \
            "Gagnant_ou_pas": score, "Service": service, "controle_player_1": controle_player1, "controle_player_2": controle_player2, \
            "attaque_player_1": attaque_player1, "attaque_player_2": attaque_player2, "poussette_player_1": poussette_player1, \
            "poussette_player_2": poussette_player2, "M1_envoie": M1_send, "M1_reception": M1_reception, \
            "M2_envoie": M2_send, "M2_reception": M2_reception, "M3_envoie": M3_send, \
            "M3_reception": M3_reception, "D1_envoie": D1_send, "D1_reception": D1_reception, "D2_envoie": D2_send, \
            "D2_reception": D2_reception, "D3_envoie": D3_send, "D3_reception": D3_reception, "G1_envoie": G1_send, \
            "G1_reception": G1_reception, "G2_envoie": G2_send, "G2_reception": G2_reception, "G3_envoie": G3_send, \
            "G3_reception": G3_reception, "Gagnant": win}, ignore_index=True)
        
    return player_data

## Déscription de la fonction permettant de formater les données

Cette fonction permet de construire un tableau pour un joueur donné, où chaque ligne correspond à un point joué. Pour chaque point seront detaillés les attributs suivant:
- la __durée__ du point selon un intervalle (si le point a été de: __courte__, __moyenne__ ou __longue__ durée) $\Rightarrow$ colonne <u>Duree</u>;
- nb de __coups droits__ du jouer selectionné lors du point $\Rightarrow$ colonne <u>coup_droit_player_1</u>:
    - de même pour l'autre joueur <u>coup_droit_player_2</u>;
- nb de coups __revers__ du jouer  selectionné lors du point $\Rightarrow$ colonne <u>revers_player_1</u>:
    - de même pour l'autre joueur <u>revers_player_2</u>;
- nb de __coups effectués__ (par l'ensemble des joueurs) $\Rightarrow$ colonne <u>Nb_coups</u>;
- numéro du __set__ dans lequel le point est joué (au total 5 set) $\Rightarrow$ colonne <u>Set</u>;
- la __progression du set__, si après le point considéré, le joueur selectionné est en train de remporter ou pas le set ('__<__': s'il est en train de perdre le set, '__>__': s'il est en train de gagner le set, '__=__': si les deux joueur ont le même score) $\Rightarrow$ colonne <u>Gagnant_ou_pas</u>;
- qui a effectué le __service__ et de quel côté $\Rightarrow$ colonne <u>Service</u>;
- nb de fois que le joueur selectionné a joué en __controle__ $\Rightarrow$ colonne <u>controle_player_1</u>:
    - de même pour l'autre joueu, colonne <u>controle_player_2</u>;
- nb de fois que le joueur selectionné a joué en __attaque__ $\Rightarrow$ colonne <u>attaque_player_1</u>:
    - de même pour l'autre joueu, colonne <u>attaque_player_1</u>;
- nb de fois que le joueur selectionné a joué avec une __poussette__ $\Rightarrow$ colonne <u>poussette_player_1</u>:    
    - de même pour l'autre joueu, colonne <u>poussette_player_2</u>;
- nb de fois que le jouer __a envoyé__ la balle dans la zone __M1__ lors du point $\Rightarrow$ colonne <u>M1_envoie</u>:
    - de même pour les autres zones de jeu $\Rightarrow$ colonnes <u>M2_envoie</u>, <u>M3_envoie</u>, etc;
- nb de fois que le jouer __a reçu__ la balle dans la zone __M1__ lors du point $\Rightarrow$ colonne <u>M1_reception</u>:     
    - de même pour les autres zones de jeu $\Rightarrow$ colonnes <u>M2_reception</u>, <u>M3_reception</u>, etc;

## <u>Pour le joueur F.T.</u>

In [13]:
ft_data = df_player("F", data)

In [14]:
ft_data

,Duree,coup_droit_player_1,coup_droit_player_2,revers_player_1,revers_player_2,Nb_coups,Set,Gagnant_ou_pas,Service,controle_player_1,...,D2_reception,D3_envoie,D3_reception,G1_envoie,G1_reception,G2_envoie,G2_reception,G3_envoie,G3_reception,Gagnant
0,long,1,1,3,2,8,1,<,player1 - Latéral droit,1,...,0,0,0,0,0,0,0,1,1,No
1,short,2,2,1,0,5,1,<,player1 - Latéral droit,1,...,0,0,1,0,0,0,0,0,1,No
2,middle,0,3,2,0,5,1,<,player2 - Latéral gauche,0,...,0,0,0,0,0,0,0,1,1,Yes
3,middle,1,1,0,1,3,1,=,player2 - Latéral droit,0,...,0,0,0,0,0,0,0,0,0,Yes
4,short,1,1,1,0,3,1,<,player1 - Latéral droit,1,...,0,0,0,0,0,0,0,0,1,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,middle,1,0,1,2,4,5,>,player1 - Latéral droit,0,...,0,1,0,1,0,0,0,0,1,Yes
103,middle,2,1,0,1,4,5,>,player2 - Latéral droit,0,...,0,0,1,0,0,0,0,0,0,No
104,middle,1,3,1,0,5,5,>,player2 - Latéral droit,0,...,0,0,0,0,0,0,0,0,0,No
105,middle,2,1,1,0,4,5,>,player1 - Latéral droit,1,...,0,0,0,0,0,0,0,0,1,No


In [19]:
target_columns = ['Gagnant']
clf = emm.EMM(width=10, depth=2, evaluation_metric='distribution_cosine')
clf.search(ft_data, target_cols=target_columns)
clf.visualise(subgroups=8, cols=3)

Sur __106__ points joués, le joueur __F__ en gagne __60__.

1. Si __F__ joue de revers et que l'adversaire ne répond pas avec un revers, il a très peu de chance de remporter le point (14 points perdus sur 16);
2. Si pendant le point, __F__ joue sur la zone __M2__ et que l'adversaire pendant le même point ne joue jamais des coup de type "contrôle", le jouer __F__ n'a pas de chance de remporter le point (9 points perdus sur 9);
3. Si __F__ est en train de perdre le set et qu'il ne reçoit jamais la balle en __M1__, il a peu de chance de remporter le point (16 points perdus sur 20);
4. si __F__ joue un coup de type "contrôle" et que le point a une durée moyenne, il a aucune chance de gagner le point (6 points perdus sur 6);
5. si pendant le point __F__ joue un coup de "revers" mais qu'il envoie jamais sur __D3__, il a peu de chances de remporter le point (16 points perdus sur 19);
6. si __F__ joue deux fois un coup de type "contrôle__ et qu'il n'envoie jamais sur __M1__, il a peu de chances de gagner le point (7 points perdus sur 8);
7. si pendant un point __F__ joue de revers et que l'adversaire ne joue aucun coup de type "poussette", __F__ a très peu de  chances de remporter le point (21 points perdus sur 29).

## <u>Pour le joueur B.T.</u>

In [16]:
bt_data = df_player("B", data)

In [17]:
# bt_data

In [20]:
target_columns = ['Gagnant']
clf = emm.EMM(width=10, depth=2, evaluation_metric='distribution_cosine')
clf.search(bt_data, target_cols=target_columns)
clf.visualise(subgroups=8, cols=3)

Le joueur __B__ a gagné __46__ points sur __106__ joués.

1. si l'adversaire a joué un coup de revers et que __B__ ne joue aucun coup de revers, __B__ a des fortes chances de gagner le point (14 points gagnés sur 16);
2. si __B__ reçoie sur __M2__ une fois lors du point et qu'il effectue jamais un coup de type "contrôle", il a des fortes chances de gagner le point (9 points gagnés sur 9);
3. si __B__ est en train de ganger le set et que il envoie jamais sur __M1__, il a des fortes chances de gagner le point (16 points gagnés sur 20);
4. si l'adversaire joue de fois un coup de type "contrôle" et que le point a une durée moyenne, __B__ a des fortes chances de gagner le point (6 points gagnés sur 6);
5. si pendant un point l'adversaire joue un seul coup de revers et qu'il envoie jamais sur __D3__,le joueur __B__ a des bonnes chances de remporter le point (26 points gagnés sur 19);
6. si l'adversaire joue deux coups de type "contrôle" lors du points et qu'il envoie jamais sur __M1__, __B__ a des fortes chances de gagner le point (7 points gagnés sur 8);
7. si l'adversaire joue un coup de revers et que __B__ ne joue aucun coup de type "poussette" lors du point, alors il a des bonnes chances de gagner le point (21 points gagnés sur 29).

## Considérations sur l'algoritmhe _EMM_

D'un point de vue technique, il n'est pas de facile implementation, étant le package pauvre en documentation. Pour en comprendre certains aspects, nous avons dû aller regarder le code source, notamment pour savoir le nombre de targets nécessité par chaque métrique (important pour appliquer une stratégie plutôt qu'une autre) ou encore le type que les colonnes sont censées avoir (type: <u>object</u>).

En dehors de ça, on peut vraiment remarquer comme l'algorithme fasse une _recherche excpetionnelle de motifs_; pour le joueur __F__ il nous propose uniquement des motifs dans lesquels il perd des points, alors qu'on sait _a priori_ qu'il est le joueur qui a remporté le plus de points et qui a gagné la partie. Viceversa, en considérant le joueur __B__, il nous retourne que des motifs pour lesquels il gagne des points.

La métrique utilisée, _distribution cosine_ permet évaluer la similarité d'objets en régardant leurs taille.